In [2]:
#code taken from https://github.com/madelinehayes/seabirdNET/blob/master/uas_img_handler_FINAL.ipynb

In [3]:
from PIL import Image
import os
from os import walk
import argparse
import numpy as np
import json
import csv
import rasterio
from rasterio.plot import reshape_as_image
from rasterio.windows import Window
import matplotlib
import folium
from pyproj import Proj, transform

%matplotlib inline

Image.MAX_IMAGE_PIXELS = 100000000000

In [4]:
def processImage(seal, image, overlap):
    '''
    args:
    seal is a string, either "Harbor_Seals" or "Elephant_Seals"
    image is the satellite image file
    overlap is an int with the size of overlap between images
    
    '''
    infile = '../Images/' + seal +'/' + image
    img_dir = '../Images/' + seal + '_Tile'
    prj_name = img_dir.split("/")[-1]
    
    #open file (read), get dims
    dataset = rasterio.open(infile)
    img_height, img_width = dataset.shape
    
    # define tile size and number of pixels to move in each direction
    tile_height = tile_width = 500
    stride = tile_height - overlap
    start_num = 0
    img_dict = {}
    count = 0
    
    # create the dir if it doesn't already exist
    if not os.path.exists(img_dir):
        os.makedirs(img_dir)
        
    num_tiles = (img_width * img_height) / ((tile_height - overlap) * (tile_width - overlap))
    print("image size", dataset.shape)
    print("number of tiles:", num_tiles)
    print("image height: ", img_height)
    print("max height", img_height-tile_height+1)
    print("image width: ", img_width)
    print("max width", img_width-tile_width+1)
    
    for r in range(0, img_height-tile_height+1, stride):
        print(r)
        for c in range(0, img_width-tile_width+1, stride):
            # read tile
            #print(c, r)
            tile = dataset.read((1,2,3),window=Window(c, r, tile_width, tile_height))
            
            reshaped_tile = reshape_as_image(tile)
            
            top_pixel = [c,r]
            name = image.split(".")[0]
            tile_name = name + "---" + str(count) + ".png"
            img_dict[tile_name] = top_pixel
            
            im = Image.fromarray(reshaped_tile)
            #save 
            path = os.path.join(img_dir, tile_name)
            im.save(path)
            
            count += 1
            
    '''
    not sure if the following is necessary...
    
    full_dict = {"image_name" : infile,
            "image_locations" : img_dict,
             "crs" : str(dataset.crs)
            }
    with open(img_dir + '/lastLine.json', 'w') as fp:
        json.dump(full_dict, fp)
    '''

In [10]:
image = files[0]
processImage(seal, image, overlap)

image size (3352, 4800)
number of tiles: 83.10743801652893
image height:  3352
max height 2853
image width:  4800
max width 4301
0
440
880
1320
1760
2200
2640


In [6]:
seal = 'Harbor_Seals'
#seal = 'Elephant_Seals'

sealPath = '../Images/' + seal

if seal == 'Harbor_Seals':
    overlap = 60
elif seal == 'Elephant_Seals':
    overlap = 80

In [7]:
files = next(walk(sealPath), (None, None, []))[2]  # empty if no files exist
#print(files)

In [8]:
'''
countI = 1
for image in files:
    processImage(seal, image, overlap)
    print('finished image', countI)
    countI += 1
'''

"\ncountI = 1\nfor image in files:\n    processImage(seal, image, overlap)\n    print('finished image', countI)\n    countI += 1\n"

In [9]:
def checkImageSize(seal, image):
    infile = '../Images/' + seal +'/' + image
    dataset = rasterio.open(infile)
    img_height, img_width = dataset.shape
    return img_height, img_width
    
    
sizes = set()
for image in files:
    size = checkImageSize(seal, image)
    sizes.add(size)
    
print(len(sizes))

1


C:\Users\mgiff\anaconda3\envs\rasterio\lib\site-packages\rasterio\__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
